In [ ]:
import tensorflow as tf
import keras
from keras import backend as k
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb 
from skimage.transform import resize
from skimage import io, color
import numpy as np
import os
import random
import shutil
import random
import glob
from google.colab import drive
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from matplotlib import pyplot as plt

In [ ]:
drive.mount('content/') 

Mounted at content/


In [ ]:
!unzip "/content/content/My Drive/Colab Notebooks/image-coloring-dataset.zip" -d "/content/trainingdata/"

In [ ]:
!pwd

/content


In [ ]:
cd /content/trainingdata/image-coloring-dataset/

/content/trainingdata/image-coloring-dataset


In [ ]:
os.makedirs('train')
os.makedirs('test')

for c in random.sample(glob.glob('*.*'), 11000):
  shutil.move(c,'train')

for c in random.sample(glob.glob('*.*'), 944):
  shutil.move(c,'test')


In [ ]:
cd /content/trainingdata/image-coloring-dataset/train/

/content/trainingdata/image-coloring-dataset/train


In [ ]:
os.makedirs('t1')                                 
os.makedirs('t2')
os.makedirs('t3')
os.makedirs('t4')
os.makedirs('t5')
os.makedirs('t6')
os.makedirs('t7')
os.makedirs('t8')
os.makedirs('t9')
os.makedirs('t10')
os.makedirs('t11')

In [ ]:
for c in random.sample(glob.glob('*.*'), 1000):            #dividing train data to 11 folders
  shutil.move(c,'t1')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'t2')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'t3')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'t4')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'t5')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'t6')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'t7')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'t8')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'t9')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'t10')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'t11')

In [ ]:
cd /content/trainingdata/image-coloring-dataset/train/t1

/content/trainingdata/image-coloring-dataset/train/t1


In [ ]:
os.makedirs('img')

for c in random.sample(glob.glob('*.*'), 1000):
  shutil.move(c,'img')

In [ ]:
cd /content/trainingdata/image-coloring-dataset/train/

/content/trainingdata/image-coloring-dataset/train


In [ ]:
path = 't1/'                  #manual selection of folder

In [ ]:
train_datagen = ImageDataGenerator(rescale= 1. / 255)

In [ ]:
train = train_datagen.flow_from_directory(path, target_size=(256,256), batch_size = 1000, class_mode=None)

Found 1000 images belonging to 1 classes.


In [ ]:
X_t = []
Y_t = []

In [ ]:
def PrepInput():                    #preprocess data
  X = []
  Y = []
  for img in train[0]:
    try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0])
      Y.append(lab[:,:,1:]/128)
    except:
      print('error')
  X = np.array(X)
  Y = np.array(Y)
  X = X.reshape(X.shape+(1,))
  print(X.shape)
  print(Y.shape)
  return X,Y

In [ ]:
X_t, Y_t = PrepInput()

(1000, 256, 256, 1)
(1000, 256, 256, 2)


In [ ]:
#model = keras.models.load_model('/content/trainingdata/image-coloring-dataset/train/imgcol.h5')  #for loading weights of last training

# **Model**

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3,3), activation ='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3,3), activation ='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation ='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation ='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation ='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation ='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation ='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation ='relu', padding='same'))

model.add(Conv2D(128, (3,3), activation ='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(64, (3,3), activation ='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation ='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation ='relu', padding='same'))
model.add(Conv2D(2, (3,3), activation ='tanh', padding='same'))
model.add(UpSampling2D((2,2)))

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 256)       590080    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 512)       1180160   
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 512)       2

In [ ]:
model.fit(X_t, Y_t, validation_split=0.1, epochs=100, batch_size=50, verbose=2)

In [ ]:
model.save('imgcol.h5')

In [ ]:
cd /content/trainingdata/image-coloring-dataset/

/content/trainingdata/image-coloring-dataset


# ***To test the model***

In [ ]:
img1_color = []
img1 = img_to_array(load_img('/content/trainingdata/image-coloring-dataset/test/1008.jpg'))    #img from test set
img1 = resize(img1, (256, 256))
img1_color.append(img1)
img1_color = np.array(img1_color, dtype=float)
img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
img1_color = img1_color.reshape(img1_color.shape+(1,))
output1 = model.predict(img1_color)
output1 = output1*128
result = np.zeros((256, 256, 3))
result[:,:,0] = img1_color[0][:,:,0]
result[:,:,1:] = output1[0]
io.imshow(lab2rgb(result))